# Complex form

In [16]:
import numpy as np
import random

# For crosscorelation
from scipy import signal
from scipy import misc

# for exporting
import codecs, json

In [2]:
# Numpy usage here is totally optional, you can use plain list comprehension like this [ L[i:i+3] for i in range(0, 9, 3)]
# * is our padding/wildcard character that matches with every character

CROSS = np.reshape(list('F*F*X*F*F'), (3, 3))
BEND  = np.reshape(list('F**F**XXX'), (3, 3))
ARROW = np.reshape(list('S*WSW*SSS'), (3, 3))

P = {'CROSS': CROSS, 'BEND': BEND, 'ARROW': ARROW}

In [3]:
def sample_input(N, M) -> (np.ndarray, int, int, np.ndarray):
  # Generate random Matrix
  vfunc = np.vectorize(lambda x: chr(x))
  output = np.random.randint(97, 122+1, size=(N, M))
  output = vfunc(output)
  
  # Select a random pattern to put inside the Matrix
  rkey = random.choice(list(P.keys()))
  selected_pattern = P[rkey]
  sX, sY = selected_pattern.shape
  
  startX = random.randint(0, N-sX)
  startY = random.randint(0, M-sY)
  
  # Put a pattern inside random Matrix
  output[startX:startX+sX, startY:startY+sY] = selected_pattern
  
  return output, startX, startY, rkey

In [4]:
A = sample_input(4, 5)
print(A[0])
print(A[-3:-1])
print(A[-1])

[['o' 'm' 'b' 'u' 's']
 ['v' 'S' '*' 'W' 'y']
 ['w' 'S' 'W' '*' 'z']
 ['y' 'S' 'S' 'S' 'c']]
(1, 1)
ARROW


In [5]:
# Generating some testcases
random_shapes = [(random.randint(3,100), random.randint(3,100)) for _ in range(100)]
TC = [sample_input(x, y) for x, y in random_shapes]

In [6]:
# Preprocessing Helpers

def preprocess_pattern(inpattern) -> np.ndarray:
  output = inpattern.copy()
  vfunc = np.vectorize(lambda x: 0 if x == '*' else ord(x))
  output = vfunc(output).astype(np.int32)
  return output

def pattern_match_number(pattern) -> int:
  return np.sum(pattern**2)

def preprocess_input(matrix) -> np.ndarray:
  vfunc = np.vectorize(lambda x: ord(x))
  return vfunc(matrix).astype(np.int32)

In [7]:
def elementwise_check(sliceM, pattern) -> bool:
  sliceM = sliceM.reshape(-1)
  pattern = pattern.reshape(-1)
  return all([True if (y == 0) else x==y for x, y in zip(sliceM, pattern)])
  
def pattern_matching(matrix) -> (bool, list):
  
  matched = False
  matched_list = []
  matrix = preprocess_input(matrix)
  
  for pname in P:
    
    current_pattern = preprocess_pattern(P[pname])
    match_key = pattern_match_number(current_pattern)
    
    c2d = signal.correlate2d(matrix, current_pattern, mode='same')
    candidates = zip(*np.where(c2d==match_key))
    
    for cd in candidates:
      x, y = cd
      w, h = current_pattern.shape
      sliceM = matrix[x-1:x-1+w, y-1:y-1+h]
      if (elementwise_check (sliceM, current_pattern)):
#         print("Found a Match")
        matched_list.append([pname, (x-1, y-1)])
        matched = True
  
  return matched, matched_list

In [8]:
myTC = sample_input(250,250)
print(pattern_matching(myTC[0]))
print(myTC)

(True, [['BEND', (169, 230)]])
(array([['p', 's', 't', ..., 'z', 'y', 'v'],
       ['y', 'r', 'k', ..., 'w', 'x', 'q'],
       ['l', 'h', 'e', ..., 'o', 'k', 'r'],
       ...,
       ['f', 'y', 'a', ..., 'q', 'x', 'v'],
       ['w', 'q', 'y', ..., 'q', 'l', 'd'],
       ['o', 'w', 'z', ..., 'j', 'j', 'e']], dtype='<U1'), 169, 230, 'BEND')


In [9]:
# does it work ?
[[sample[-1] ,pattern_matching(sample[0])] for sample in TC]

[['BEND', (True, [['BEND', (23, 47)]])],
 ['ARROW', (True, [['ARROW', (9, 10)]])],
 ['CROSS', (True, [['CROSS', (10, 6)]])],
 ['CROSS', (True, [['CROSS', (0, 44)]])],
 ['ARROW', (True, [['ARROW', (44, 30)]])],
 ['CROSS', (True, [['CROSS', (37, 10)]])],
 ['ARROW', (True, [['ARROW', (9, 11)]])],
 ['BEND', (True, [['BEND', (2, 34)]])],
 ['ARROW', (True, [['ARROW', (15, 17)]])],
 ['ARROW', (True, [['ARROW', (35, 2)]])],
 ['ARROW', (True, [['ARROW', (0, 9)]])],
 ['BEND', (True, [['BEND', (26, 10)]])],
 ['ARROW', (True, [['ARROW', (2, 59)]])],
 ['ARROW', (True, [['ARROW', (1, 24)]])],
 ['BEND', (True, [['BEND', (1, 7)]])],
 ['ARROW', (True, [['ARROW', (7, 25)]])],
 ['BEND', (True, [['BEND', (33, 30)]])],
 ['ARROW', (True, [['ARROW', (25, 49)]])],
 ['ARROW', (True, [['ARROW', (65, 1)]])],
 ['CROSS', (True, [['CROSS', (13, 12)]])],
 ['CROSS', (True, [['CROSS', (14, 10)]])],
 ['CROSS', (True, [['CROSS', (62, 18)]])],
 ['ARROW', (True, [['ARROW', (23, 53)]])],
 ['CROSS', (True, [['CROSS', (86, 3

# Simpler form

In [10]:
CROSS = np.reshape(list('F.F.X.F.F'), (3, 3))
BEND  = np.reshape(list('F..F..XXX'), (3, 3))
ARROW = np.reshape(list('S.WSW.SSS'), (3, 3))

P = {'CROSS': CROSS, 'BEND': BEND, 'ARROW': ARROW}

In [11]:
def sample_input(N, M) -> (np.ndarray, int, int, np.ndarray):
  # generating a placeholder matrix filled with pad character '.'
  output = np.repeat('.', N*M).reshape(N, M)
  
  # Select a random pattern to put inside the Matrix
  rkey = random.choice(list(P.keys()))
  selected_pattern = P[rkey]
  sX, sY = selected_pattern.shape
  
  startX = random.randint(0, N-sX)
  startY = random.randint(0, M-sY)
  
  # Put a pattern inside random Matrix
  output[startX:startX+sX, startY:startY+sY] = selected_pattern
  
  return output, startX, startY, rkey

In [12]:
def elementwise_check(sliceM, pattern) -> bool:
  if (sliceM.shape != pattern.shape):
    return (False, '')
  
  sliceM = sliceM.reshape(-1)
  pattern = pattern.reshape(-1)
  return all([True if (y == 0) else x==y for x, y in zip(sliceM, pattern)])
  
def pattern_matching(matrix) -> (bool, str):
  # clear up th input matrix
  matrix = matrix[~np.all(matrix=='.', axis=1)] # Removing empty rows
  matrix = matrix[:, ~np.all(matrix=='.', axis=0)] # Removing empty columns
  
  matched_list = []
  
  for pkey in P:
    if elementwise_check(matrix, P[pkey]): matched_list.append(pkey)
    
  return (True, matched_list[0]) if len(matched_list) == 1 else (False, '')

In [13]:
random_shapes = [(random.randint(3,100), random.randint(3,100)) for _ in range(100)]
TC = [sample_input(x, y) for x, y in random_shapes]

In [14]:
# test against all test cases!
np.all([pattern_matching(x)[1] == w for x, _, _, w in TC])

True